## Importing required libraries

In [61]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pandas_profiling import ProfileReport
import re 
import warnings
warnings.filterwarnings("ignore")

## Loading and viewing data

In [62]:
met_small = pd.read_csv('Data/movies_metadata.csv')

In [63]:
met_small.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Explantory Data Analysis

In [64]:
# Extract the columns of interest and replace the NaN with blank string

movies_data = met_small[['title','genres','overview','production_companies','original_language','status','vote_count','vote_average']].replace(np.nan, '', regex=True)

In [65]:
#ProfileReport(movie_dataset)

## Data Cleaning

In [66]:
#Remove movies with less than 5 imdb rating and langauges other than english and hindi

#First replace Empty values "" with 0
movies_data['vote_average']=movies_data['vote_average'].replace("",0)

#movies with rating more than 5 will be included in the list
movie_more5 = movies_data[movies_data['vote_average']>5]

#Only released movies to be included in the list
movie_rel = movie_more5[movie_more5['status']=='Released']

#Convert the title to lowercase
movie_rel['title'] = movie_rel['title'].str.lower()

#Select the major languages
movie_lang = movie_rel[movie_rel['original_language'].isin(['en','ja','it','de','es','ru','ko','hi'])== True]

#Select movies with more than 500 vote counts
movie_vote = movie_lang[movie_lang['vote_count']>500]

#Reset index after removal of rows
movie_dataset = movie_vote.reset_index().drop(columns='index',axis=1)

In [67]:
#Define a function to combine all strings

def combine_features(row):
    try:
        return row['title'] +2*(" "+row["genres"])+" "+row["overview"]+" "+row["production_companies"]+3*(" "+row["original_language"])
    except:
        print("Error:",row)

In [68]:
#Combine title, genre, overview and production companies
#Thus a user will be recommended movies that are similar in genre, overiew and production company

movie_dataset["combined_features"] = movie_dataset.apply(combine_features,axis=1)

#Convert the string to lower case
movie_dataset["combined_features"]= movie_dataset["combined_features"].str.lower()

#Lets view one of the combine feature
movie_dataset["combined_features"][0]

"toy story [{'id': 16, 'name': 'animation'}, {'id': 35, 'name': 'comedy'}, {'id': 10751, 'name': 'family'}] [{'id': 16, 'name': 'animation'}, {'id': 35, 'name': 'comedy'}, {'id': 10751, 'name': 'family'}] led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences. [{'name': 'pixar animation studios', 'id': 3}] en en en"

In [69]:
#Lets remove the unwanted parentheses, id, name, punctuation

movie_dataset.astype(str)['combined_features'].map(lambda x:  type(x))

replace_val = ["{'id'","'name':","'},","[","]","'",",","}]","{","}",".",":","'",'[0-9]','id','--']

for i in replace_val:
  movie_dataset["combined_features"] = movie_dataset["combined_features"].str.replace(i, '')

for i in range(len(movie_dataset)):
    movie_dataset["combined_features"][i] = re.sub(' +', ' ', movie_dataset["combined_features"][i]) 

movie_dataset["combined_features"][0]

'toy story animation comedy family animation comedy family led by woody andys toys live happily in his room until andys birthday brings buzz lightyear onto the scene afra of losing his place in andys heart woody plots against buzz but when circumstances separate buzz and woody from their owner the duo eventually learns to put ase their differences pixar animation studios en en en'

In [70]:
#Copy movie dataset to df
df= movie_dataset.copy(deep=True)

#Create a column named index
df['index'] = df.index

## Save the Pre=processed dataset

In [71]:
df

,title,genres,overview,production_companies,original_language,status,vote_count,vote_average,combined_features,index
0,toy story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",en,Released,5415,7.7,toy story animation comedy family animation co...,0
1,jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",en,Released,2413,6.9,jumanji adventure fantasy family adventure fan...,1
2,heat,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","Obsessive master thief, Neil McCauley leads a ...","[{'name': 'Regency Enterprises', 'id': 508}, {...",en,Released,1886,7.7,heat action crime drama thriller action crime ...,2
3,goldeneye,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",James Bond must unmask the mysterious head of ...,"[{'name': 'United Artists', 'id': 60}, {'name'...",en,Released,1194,6.6,goldeneye adventure action thriller adventure ...,3
4,casino,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",The life of the gambling paradise – Las Vegas ...,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",en,Released,1343,7.8,casino drama crime drama crime the life of the...,4
...,...,...,...,...,...,...,...,...,...,...
1967,what happened to monday,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",In a world where families are limited to one c...,"[{'name': 'Vendome Pictures', 'id': 7460}, {'n...",en,Released,598,7.3,what happened to monday science fiction thrill...,1967
1968,atomic blonde,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",An undercover MI6 agent is sent to Berlin duri...,"[{'name': 'Denver and Delilah Productions', 'i...",en,Released,748,6.1,atomic blonde action thriller action thriller ...,1968
1969,dunkirk,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",The miraculous evacuation of Allied soldiers f...,"[{'name': 'Canal+', 'id': 5358}, {'name': 'Stu...",en,Released,2712,7.5,dunkirk action drama history thriller war acti...,1969
1970,transformers: the last knight,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...","Autobots and Decepticons are at war, with huma...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...",en,Released,1440,6.2,transformers the last knight action science fi...,1970


In [72]:
df.to_csv("Appdata/Movie_dataset.csv")

## Create model and predict

In [79]:
#Create count matrix from this new combined column

cv = CountVectorizer()

count_matrix = cv.fit_transform(df["combined_features"])

#Compute the Cosine Similarity based on the count_matrix

cosine_sim = cosine_similarity(count_matrix) 
movie_user_likes = "heat"

In [80]:
#Helper functions to obtain index of movie from its title

def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

In [81]:
#Get index of this movie from its title

movie_index = get_index_from_title(movie_user_likes)

similar_movies =  list(enumerate(cosine_sim[movie_index]))


#Get a list of similar movies in descending order of similarity score

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

# Print titles of first 5 movies
i=0
for element in sorted_similar_movies:
  if get_title_from_index(element[0])==movie_user_likes:
    pass
  else:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
      break

mystic river
a most violent year
shooter
zodiac
l.a. confidential
point break
